## Work
1. 請嘗試寫一個 callback 用來記錄各類別在訓練過程中，對驗證集的 True Positive 與 True Negative

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    input_layer = keras.layers.Input([x_train.shape[-1]])
    
    for index,neuron in enumerate(num_neurons):
        if(index == 0):
            x = keras.layers.Dense(units = neuron,activation = 'relu')(input_layer)
        else:
            x = keras.layers.Dense(units = neuron,activation = 'relu')(x)
        x = BatchNormalization()(x)
    output_layer = keras.layers.Dense(units = output_units,activation = 'softmax')(x)
    model = keras.models.Model(inputs = [input_layer],outputs = [output_layer])
    
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [10]:
# 載入 Callbacks
from keras.callbacks import Callback
import numpy as np
# Record_fp_tp
class Record_tp_tn(Callback):
    
    def __init__(self,val_data):
        self.val_data = val_data
    """Code Here
    撰寫一個紀錄 ture positive(分類為TRUE且正確的數量 = validation_set.) 與 true negative(分類為False且正確的數量) 數量的 callbacks 
    ?? f1_score 只能拿來計算binary classification不是嗎
    """
    
    def on_train_begin(self,epoch,logs = {}):
        if('metrics' in self.params):
            if('f1_score' not in self.params['metrics']):
                self.params['metrics'].append('val_f1_score')
            
    def on_epoch_end(self,epoch,logs = {}):
        logs = logs or {}
        y_true = self.val_data[1]
        y_pred = self.model.predict(self.val_data[0])
        
        #true_positive = y_pred[np.where(y_pred[np.where(y_true == 1)] == 1)]
        #print(len(true_positive))
        
        #true_negative = y_pred[np.where(y_pred[np.where(y_true == 0)] == 0)]
        #print(len(true_negative))
        
        #logs['val_true_positive'] = len(true_positive)
        #logs['val_true_negative'] = len(true_negative)
        logs['val_true_positive'] = 0
        logs['val_true_negative'] = 0
        #true_positive = (y_pred[y_true == 1] == 1)
rec_tptn = Record_tp_tn((x_test,y_test))

In [11]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

"""Code Here
將 callbacks 放入模型訓練流程中
"""
model.fit(x_train,y_train,
         epochs = EPOCHS,
         batch_size = BATCH_SIZE,
         validation_data = (x_test,y_test),
         shuffle = True,
         callbacks = [rec_tptn])

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

"""Code Here
將 tp/tn 從訓練完的模型中取出
"""
valid_tp = model.history.history['val_true_positive']
valid_tn = model.history.history['val_true_negative']

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               786688    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_6 (Batch (None, 256)               1024

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

plt.plot(range(len(valid_tp)), valid_tp, label="valid tp", color="navy")
plt.plot(range(len(valid_tn)), valid_tn, label="valid tn", color="red")
plt.legend()
plt.title("True positives and True Negatives")
plt.show()
